In [1]:
!pip install openml==0.13.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 125.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 15.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.


In [1]:
import openml
import os

In [2]:
# Download the dataset. You can ignore the warning.
d = openml.datasets.get_dataset(44282, download_all_files=True)
df, *_ = d.get_data()

# Defines where the raw image files are stored
# OpenML caches them by default, so we can point there
data_path = os.path.join(openml.config.cache_directory, "org/openml/www/datasets/44282/PLK_Mini/images/")

/home/scur0846/.local/lib/python3.10/site-packages/openml/datasets/functions.py:404: UserWarning: ``download_all_files`` is experimental and is likely to break with new releases.
  warnings.warn(


In [5]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, stratify = df['CATEGORY'], test_size = 0.1, random_state=420)

In [6]:
!mkdir ./data/PLK_Mini/train/
!mkdir ./data/PLK_Mini/test/

In [7]:
import torchvision

In [8]:
train_path = './data/PLK_Mini/train/'
test_path = './data/PLK_Mini/test/'

In [9]:
for index, row in df_train.iterrows():
    img = torchvision.io.read_image(os.path.join(data_path, row['FILE_NAME']))
    categ_dir = os.path.join(train_path, row['CATEGORY'])
    if not os.path.isdir(categ_dir):
        os.mkdir(categ_dir)
    torchvision.io.write_png(img, os.path.join(categ_dir, row['FILE_NAME']))

In [10]:
for index, row in df_test.iterrows():
    img = torchvision.io.read_image(os.path.join(data_path, row['FILE_NAME']))
    categ_dir = os.path.join(test_path, row['CATEGORY'])
    if not os.path.isdir(categ_dir):
        os.mkdir(categ_dir)
    torchvision.io.write_png(img, os.path.join(categ_dir, row['FILE_NAME']))

### Train and evaluate with default  (768) number of prototypes

In [ ]:
!python3 main.py --dataset plankton --epochs_pretrain 10 --batch_size 16 --batch_size_pretrain 16 --validation_size 0.2 --image_size 128 --freeze_epochs 0 --epochs 60 --log_dir ./runs/plankton_258_protos 

In [ ]:
!python3 main.py --dataset plankton --epochs_pretrain 0 --batch_size 16 --freeze_epochs 0 --epochs 0 --log_dir ./runs/plankton_test_2023_11_11_1530_768_protos_full_vis --image_size 128 --state_dict_dir_net ./runs/plankton_258_protos/checkpoints/net_trained_60

### Train  and evaluate with 258 prototypes

In [ ]:
!python3 main.py --dataset plankton --epochs_pretrain 10 --batch_size 16 --batch_size_pretrain 16 --validation_size 0.2 --image_size 128 --freeze_epochs 0 --epochs 60 --log_dir ./runs/plankton_258_protos --num_features 258 

In [5]:
!python3 main.py --dataset plankton --epochs_pretrain 0 --batch_size 16 --freeze_epochs 0 --epochs 0 --log_dir ./runs/plankton_test_2023_11_11_1530_768_protos_full_vis --image_size 128 --num_features 258 --state_dict_dir_net ./runs/plankton_258_protos/checkpoints/net_trained_60